In [1]:
import pandas as pd
import os
import seaborn as sns

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

import shutil

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime as dt

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

import time

# ** Librerías propias
import funcion_parametrizacion_variables
import funcion_SQL_a_DataFrame
import funcion_exportacion_pdf_seriemapas

In [2]:
resultado = funcion_parametrizacion_variables.parametrizacion_variables()
consulta_tRendimientosActualizacion = resultado[2]
renombrar_actividades = resultado[4]
consulta_informalidades = resultado[12]
dict_area_municipio = resultado[13]
consulta_interesados = resultado[14]

In [3]:

fecha_actual = dt.now()
fecha_directorio = str(fecha_actual.strftime("%Y%m%d"))

FC_TERRENOS_UNIFICADOS = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada\TERRENO_POR_HITO"

RUTA_BDLOCAL = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_9_Indicador_Juridico\Base_Datos\2_2_9_IndicadorJuridico.gdb"
NOMBRE_CAPA = 'terrenos_matrices_con_informales'
NOMBRE_CAPA_AVANCE_COMPONENTEJURID = 'predios_con_avanceJuridico'

# TODO: Generación XLSX Resultados
DIRECTORIO_XLSX = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_9_Indicador_Juridico\zReportes"
NOMBRE_XLSX = fecha_directorio+'_Seguimiento_indicador_juridico.xlsx'

# TODO: Generación Salidas Gráficas
DIRECTORIO_PDF = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_9_Indicador_Juridico\Salidas_Graficas"


In [4]:

df_consulta_tRendimientosActualizacion = funcion_SQL_a_DataFrame.sql_a_dataframe(consulta_tRendimientosActualizacion)
df_consulta_informalidades = funcion_SQL_a_DataFrame.sql_a_dataframe(consulta_informalidades)
df_consulta_interesados = funcion_SQL_a_DataFrame.sql_a_dataframe(consulta_interesados)

for columna in df_consulta_tRendimientosActualizacion.columns:
    if columna == 'actividad_actual_tramite':
        for llave, valor in renombrar_actividades.items():
            df_consulta_tRendimientosActualizacion.loc[df_consulta_tRendimientosActualizacion[columna]==llave, columna] = valor

Inicio de la conexión
Conflicto detectado, reintentando en 10 segundos...
Inicio de la conexión
Conexión exitosa ...
Inicio de la conexión
Conexión exitosa ...
Inicio de la conexión
Conexión exitosa ...


In [5]:

# TODO: Llamadas a las variables
df_terrenos_unificados = pd.DataFrame.spatial.from_featureclass(FC_TERRENOS_UNIFICADOS)

<h3>Estadísticos de Avance</h3>

In [6]:

# TODO: Se filtran todas las actividades que ya hallan pasado por el análisis Catastro Registro
df_consulta_tRendimientosActualizacion_ajuste_juridico = df_consulta_tRendimientosActualizacion[(df_consulta_tRendimientosActualizacion['actividad_actual_tramite'] != '1.Incorporando predios a la versión') & 
    (df_consulta_tRendimientosActualizacion['actividad_actual_tramite'] != '2.Realizando Análisis Catastro Registro')][['radicado','actividad_actual_tramite','npn','municipio','ficha','tipo_terreno']]

# TODO: Como los NPN se duplican por el cruce con las informalidades, se debe dejar un único NPN
df_consulta_tRendimientosActualizacion_ajuste_juridico_unicos = df_consulta_tRendimientosActualizacion_ajuste_juridico.drop_duplicates()
print(f"Total de registros existentes: {df_consulta_tRendimientosActualizacion_ajuste_juridico_unicos.shape[0]}")

# TODO: Se carga la capa de avance del indicador
ruta_fc_indicador = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_9_Indicador_Juridico\Base_Datos\2_2_9_IndicadorJuridico.gdb\predios_con_avanceJuridico"
if arcpy.Exists(ruta_fc_indicador):
    df_fc_base = pd.DataFrame.spatial.from_featureclass(ruta_fc_indicador)
    df_fc_base = df_fc_base['numero_predial_nacional']
    df_fc_base = df_fc_base.drop_duplicates()

else:
    print(f"La capa no existe")

# TODO: Se cruza la capa de avance del indicador con la nueva consulta, buscando indentificar registros que sean nuevos y no se encuentren en la capa base del indicador
df_actualizacion_base = pd.merge(left=df_consulta_tRendimientosActualizacion_ajuste_juridico_unicos,
right=df_fc_base,
left_on='npn',
right_on='numero_predial_nacional',
how='left')

# TODO: Aquí se reconocen los registros que por Devoluciones, cambian de semana a semana
df_base_actualizacion = pd.merge(left=df_consulta_tRendimientosActualizacion_ajuste_juridico_unicos,
right=df_fc_base,
left_on='npn',
right_on='numero_predial_nacional',
how='right')

columnas = ['radicado','actividad_actual_tramite','npn','municipio','ficha','tipo_terreno']

# TODO: Se garantiza mantener los devueltos dentro de los ya análisis jurídicos realizados
df_base_actualizacion = df_base_actualizacion[df_base_actualizacion['npn'].isna()][columnas]
print(f"Total de registros asociados a Devoluciones: {df_base_actualizacion.shape[0]}")

# TODO: Por el JOIN se buscan los registros que aparezcan en el sistema que no se alojen en la base de datos geo base
df_actualizacion_base_nuevos_registros = df_actualizacion_base[df_actualizacion_base['numero_predial_nacional'].isna()][columnas]
print(f"Total de nuevos registros: {df_actualizacion_base_nuevos_registros.shape[0]}")

# TODO: Se ajusta el dataframe que contiene los registros que son antiguos, es decir, que ya aparecián en la capa base
df_actualizacion_base_registros_existentes = df_actualizacion_base[df_actualizacion_base['numero_predial_nacional'].notna()][columnas]
print(f"Total de registros existentes: {df_actualizacion_base_registros_existentes.shape[0]}")

# TODO: Se concatenan registros nuevos vs registros antiguos
df_unificado = pd.concat([df_actualizacion_base_registros_existentes, df_actualizacion_base_nuevos_registros, df_base_actualizacion]).drop_duplicates()
print(f"Total de registros unificados: {df_unificado.shape[0]}")

# TODO: Se cruza con Terrenos para conocer el Hito al que pertenen estos predios
df_ajustejurico_vs_terrenos = pd.merge(left=df_unificado,
    right=df_terrenos_unificados,
    left_on='npn',
    right_on='codigo',
    how='inner'
)

# TODO: Se genera el dataframe, con el universo consilidado, para ser cruzado con predios (buscando propietarios, derecho, etc), y que sirve de base para las nuevas incorporaciones de datos jurídicas
df_ajustejurico_vs_terrenos_completo = df_ajustejurico_vs_terrenos

# TODO: Se filtran los Informales. Si se mantuvieran lo que significa es que me reconocerá aquellos que ya tienen geometría desde SIG
# TODO: Al extraerlos se totaliza el total de Informalidades por Mpío según relación.
df_ajustejurico_vs_terrenos = df_ajustejurico_vs_terrenos[df_ajustejurico_vs_terrenos['tipo_terreno'] != 'Informal']

# TODO: Cálculo de Terrenos Ajustados Jurídicamente por Municipio (limitado por los terrenos)
df_ajustejurico_vs_terrenos_x_mpio = df_ajustejurico_vs_terrenos.groupby(['nombre_municipio']).agg({'npn': 'count', 'area_ha_cmt12': 'sum'})
df_ajustejurico_vs_terrenos_x_mpio = (pd.DataFrame(df_ajustejurico_vs_terrenos_x_mpio).reset_index()).rename(columns={'npn':'total_predios_formales', 'area_ha_cmt12':'total_area_ha_predios'})
df_ajustejurico_vs_terrenos_x_mpio['total_area_ha_predios'] = (df_ajustejurico_vs_terrenos_x_mpio['total_area_ha_predios']).round(3)

# TODO: Unificación con áreas de Municipios
# TODO: Transformación de diccionario a DF
df_areas_municipio = pd.DataFrame(list(dict_area_municipio.items()), columns=['Municipio', 'Area'])

df_ajustejurico_vs_terrenos_x_mpio_w_areampio = (pd.merge(left=df_ajustejurico_vs_terrenos_x_mpio,
    right=df_areas_municipio,
    left_on='nombre_municipio',
    right_on='Municipio',
    how='inner'
)).rename(columns={'Area':'area_ha_municipio'})

# TODO: Calculo del Avance
df_ajustejurico_vs_terrenos_x_mpio_w_areampio['porcentaje_avance_area'] = ((df_ajustejurico_vs_terrenos_x_mpio_w_areampio['total_area_ha_predios']/df_ajustejurico_vs_terrenos_x_mpio_w_areampio['area_ha_municipio']) * 100).round(3)

# TODO: Incorporación Informalidades
df_consulta_informalidades['municipio_informalidad'] = None

df_consulta_informalidades.loc[df_consulta_informalidades['numero_predial_informal'].str.startswith('08606'), 'municipio_informalidad'] = 'Repelon'
df_consulta_informalidades.loc[df_consulta_informalidades['numero_predial_informal'].str.startswith('13442'), 'municipio_informalidad'] = 'MariaLaBaja'
df_consulta_informalidades.loc[df_consulta_informalidades['numero_predial_informal'].str.startswith('08078'), 'municipio_informalidad'] = 'Baranoa'
df_consulta_informalidades.loc[df_consulta_informalidades['numero_predial_informal'].str.startswith('08634'), 'municipio_informalidad'] = 'Sabanagrande'

df_consulta_informalidades_x_municipio = df_consulta_informalidades.groupby(['municipio_informalidad'])['numero_predial_informal'].count()
df_consulta_informalidades_x_municipio = ((pd.DataFrame(df_consulta_informalidades_x_municipio)).reset_index()).rename(columns={'numero_predial_informal':'total_predios_informales'})


df_ajustejurico_vs_terrenos_x_mpio_w_areampio_x_informalidad = pd.merge(left=df_ajustejurico_vs_terrenos_x_mpio_w_areampio,
    right=df_consulta_informalidades_x_municipio,
    left_on='nombre_municipio',
    right_on='municipio_informalidad',
    how='inner'
)

# TODO: Cálculo del total de predios (predios formales + predios informales)
df_ajustejurico_vs_terrenos_x_mpio_w_areampio_x_informalidad['numero_predios'] = df_ajustejurico_vs_terrenos_x_mpio_w_areampio_x_informalidad['total_predios_formales'] + df_ajustejurico_vs_terrenos_x_mpio_w_areampio_x_informalidad['total_predios_informales']

# TODO: Reorganización de Columnas
reorganizacion_columnas = ['nombre_municipio','area_ha_municipio','total_area_ha_predios','porcentaje_avance_area','numero_predios', 'total_predios_formales', 'total_predios_informales']
df_ajustejurico_vs_terrenos_x_mpio_w_areampio_x_informalidad = df_ajustejurico_vs_terrenos_x_mpio_w_areampio_x_informalidad[reorganizacion_columnas]

with pd.ExcelWriter(os.path.join(DIRECTORIO_XLSX,NOMBRE_XLSX), engine='openpyxl') as writer:
    df_ajustejurico_vs_terrenos_x_mpio_w_areampio_x_informalidad.to_excel(writer, sheet_name='segIndJuridico', index=False)

Total de registros existentes: 9745
Total de registros asociados a Devoluciones: 92
Total de nuevos registros: 3396
Total de registros existentes: 6349
Total de registros unificados: 9746


<h3>Capa Geográfica para Informe Semanal</h3>

In [7]:

arcpy.env.overwriteOutput = True

# TODO: Se cruza con Terrenos para conocer el Hito al que pertenen estos predios
df_interesado_vs_terrenos = pd.merge(left=df_consulta_interesados,
    right=df_ajustejurico_vs_terrenos_completo,
    left_on='numero_predial_nacional',
    right_on='npn',
    how='inner'
)

# TODO: Estandarización nombres
estandarizacion_columnas = ['numero_predial_nacional','tipo_predio','orip','matricula_inmobiliaria','destino_economico','tipo_derecho','fraccion_derecho','fecha_tenencia','tipo_documento','numero_documento','nombre_interesado','fecha_visita','SHAPE']
df_interesado_vs_terrenos = df_interesado_vs_terrenos[estandarizacion_columnas]

# TODO: Importación de resultados a base de datos geográficos
df_interesado_vs_terrenos.spatial.to_featureclass(location=os.path.join(RUTA_BDLOCAL,NOMBRE_CAPA_AVANCE_COMPONENTEJURID))
print(f"Se crea capa geográfica {NOMBRE_CAPA_AVANCE_COMPONENTEJURID}")

cannot add field: 'fecha_tenencia'
cannot add field: 'fecha_visita'
Se crea capa geográfica predios_con_avanceJuridico


<h3>Espacialización de Terrenos Matrices con Informalidades</h3>

In [8]:
df_terrenos_unificados_informalidades = pd.merge(left=df_terrenos_unificados,
    right=df_consulta_informalidades,
    left_on='codigo',
    right_on='numero_predial_formal',
    how='left'
)

df_terrenos_unificados_informalidades = df_terrenos_unificados_informalidades[df_terrenos_unificados_informalidades['numero_predial_informal'].notna()]

# TODO: De los Terrenos que Tienen Formalidades, se debe dejar solo una geometría, esto mientras se accede a la BD Definitiva.
estandarizacion_terrenos_unicos = ['codigo','id_ui','meta_hito']
df_terrenos_matrices_w_informalidades = df_terrenos_unificados_informalidades[estandarizacion_terrenos_unicos]
df_terrenos_matrices_w_informalidades_unico = df_terrenos_matrices_w_informalidades.drop_duplicates()

# TODO: Total de Terrenos con Informalidades por Hito
df_terrenos_matrices_w_informalidades_unico_x_hito = df_terrenos_matrices_w_informalidades_unico['meta_hito'].value_counts()

df_terrenos_matrices_w_informalidades_unico_geo = pd.merge(left=df_terrenos_matrices_w_informalidades_unico,
    right=df_terrenos_unificados,
    on='codigo',
    how='inner'
)

df_terrenos_matrices_w_informalidades_unico_geo = df_terrenos_matrices_w_informalidades_unico_geo[['codigo','id_ui_x','meta_hito_x','area_ha_cmt12','SHAPE']]
df_terrenos_matrices_w_informalidades_unico_geo = df_terrenos_matrices_w_informalidades_unico_geo.rename(columns={'id_ui_x':'id_ui', 'meta_hito_x':'meta_hito'})

df_terrenos_matrices_w_informalidades_unico_geo.spatial.to_featureclass(location=os.path.join(RUTA_BDLOCAL,NOMBRE_CAPA))
print(f"Se crea la capa {NOMBRE_CAPA}")

Se crea la capa terrenos_matrices_con_informales


<h3>Salida Gráfica</h3>

In [9]:

# TODO: Generación Salida Gráfica
funcion_exportacion_pdf_seriemapas.exportacion_salida_grafica('Seguimiento E. Juridico', DIRECTORIO_PDF)
print(f"Se exporta Salida Gráfica")

Nombre Layout: Seguimiento E. Juridico
Se exporta a PDF Seguimiento E. Juridico
Se exporta Salida Gráfica
